In [43]:
import numpy as np 
from numpy import sin, cos
from math import atan2
import sys 
sys.path.append("../")


from pydrake.common.value import Value
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, ExternallyAppliedSpatialForce, ExternallyAppliedSpatialForceMultiplexer
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator
from pydot import graph_from_dot_data
import matplotlib.pyplot as plt


from src.LauncherFunct.PlanarPRR_launcher import PlanarPRR_Controller
from src.plant.PRR import PlanarPRR



In [3]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [15]:
class ForceControllerPD(LeafSystem):
    
    def __init__(self, plant, Kp=10, Kd=100):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", plant.num_multibody_states())
        self.DeclareVectorInputPort("target_position", BasicVector(2))
        self.DeclareVectorOutputPort("cartesianForce", BasicVector(2), self.CalcForceControl)

        self.Kp = Kp
        self.Kd = Kd
        self.link_len = 0.5

    def CalcForceControl(self, context, F):
        state = self.get_input_port(0).Eval(context)
        q = state[0], state[1]
        q_dot = state[8], state[9]
        x, y, x_dot, y_dot = self.kinematic_and_diffKin(q, q_dot)
        x_req, y_req = self.get_input_port(1).Eval(context)
        fx = self.Kp*(x_req - x) - self.Kp*(x_dot)
        fy = self.Kp*(y_req - y) - self.Kp*(y_dot)
        F.SetFromVector([fx, fy])

    def kinematic_and_diffKin(self, q, q_dot):

        x = self.link_len*(np.cos(q[0]) + np.cos([q[0] + q[1]]))
        y = self.link_len*(np.sin(q[0]) + np.sin([q[0] + q[1]]))

        x_dot = (-self.link_len*(np.sin(q[0]) + np.sin(q[0] + q[1]))*q_dot[0]) -(self.link_len*np.sin(q[0]+q[1]))*q_dot[1]
        y_dot = self.link_len*(np.cos(q[0]) + np.cos(q[0] + q[1]))*q_dot[0] + self.link_len*np.cos(q[0] + q[1])*q_dot[1]

        return x, y, x_dot, y_dot






class EndEffector_force_input_Sys(LeafSystem):
    def __init__(self, l3_index):
        LeafSystem.__init__(self)
        ## input
        self.DeclareVectorInputPort("cartesian Force", BasicVector(2))

        ##output
        self.force_obj = ExternallyAppliedSpatialForce()
        self.force_obj.body_index = l3_index
        self.force_obj.p_BoBq_B = np.array([0,0, 0.25]) # i belive that this is the translation vector to the EE

        force_cls = Value[list[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort("F-cartesian", 
                                       lambda: force_cls(), 
                                       self.CalcSpatialForce)

    def CalcSpatialForce(self, context, applied_force):
        fx, fy = self.get_input_port(0).Eval(context)
        print(fx, fy)
        F = SpatialForce(tau=np.zeros(3), f=np.array([fx, fy, 0]))
        self.force_obj.F_Bq_W = F
        applied_force.set_value([self.force_obj])


In [39]:
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModels('../sdf_model/PRR_planar.sdf')
plant.Finalize()

In [38]:
plant.get_applied_spatial_force_input_port()
controller.get_output_port(0).size()
apply_force_sys.get_input_port().size()

2

In [40]:
controller = builder.AddSystem(ForceControllerPD(plant))
apply_force_sys = builder.AddSystem(EndEffector_force_input_Sys(plant.GetBodyByName('l3').index()))

builder.Connect(apply_force_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())
builder.Connect(controller.get_output_port(), apply_force_sys.get_input_port())    
builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph.get_query_output_port(),
    meshcat
    )

diagram = builder.Build()
    

In [41]:
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

In [42]:
from pydot import graph_from_dot_data


def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/PRR_planar.svg")

In [7]:

meshcat.Delete()

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModels('../sdf_model/PRR_planar.sdf')
plant.Finalize()

planar_controller_diagram, component, planar_controller_context = PlanarPRR_Controller()
controller = builder.AddSystem(planar_controller_diagram)

builder.Connect(plant.get_output_port(), controller.get_input_port()) # q_vect
builder.Connect(controller.get_output_port(0), plant.get_input_port(0))

BigDiagram = builder.Build()

'''
root_context = BigDiagram.CreateDefaultContext()
controller_context = controller.GetMyContextFromRoot(root_context)
plant_context = plant.GetMyContextFromRoot(root_context)
cartesian_controller_context = component['cartesian controller'].GetMyContextFromRoot(root_context)

q_state = np.array([0, -0.4, 1.1, 0, 0, 0])

theta_state = mapping_q_to_theta(q_state)

plant_context.SetContinuousState(q_state)

controller_context.SetContinuousState(theta_state)
component['cartesian controller'].get_input_port_desired_state().FixValue(cartesian_controller_context, np.array([2, 0.9, 0, 0]))

'''

RuntimeError: System::get_output_port(): drake::multibody::MultibodyPlant<double> system '::plant' has 21 outputs, so this convenience function cannot be used; instead, use another overload e.g. get_output_port(OutputPortIndex) or GetOutputPort(string)